## CX Calibration with HPO

#### Imports

In [1]:
import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
module_path = os.path.abspath(os.path.join('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control'))
if module_path not in sys.path:
    sys.path.append(module_path)

from template_configurations import gate_q_env_config
import logging
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s INFO %(message)s", # hardcoded INFO level
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,
)

/Users/lukasvoss/Library/Python/3.9/lib/python/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(


Starting Rabi experiment for qubit 0...
Rabi experiment for qubit 0 done.
Starting Drag experiment for qubit 0...
Drag experiments done for qubit 0 done.
Starting Rabi experiment for qubit 1...
Rabi experiment for qubit 1 done.
Starting Drag experiment for qubit 1...
Drag experiments done for qubit 1 done.
All single qubit calibrations are done
Updated Instruction Schedule Map <InstructionScheduleMap(1Q instructions:
  q0: {'reset', 'delay', 'h', 'x', 'sx', 'sdg', 'z', 's', 'id', 'tdg', 'measure', 't', 'rz'}
  q1: {'reset', 'delay', 'h', 'x', 'sx', 'sdg', 'z', 's', 'id', 'tdg', 'measure', 't', 'rz'}
Multi qubit instructions:
  (0, 1): {'ecr', 'cr45m', 'cr45p'}
  (1, 0): {'ecr', 'cr45m', 'cr45p'}
)>


Which gate is to be calibrated?

In [2]:
gate_q_env_config.target

{'register': [0, 1],
 'gate': Instruction(name='cx', num_qubits=2, num_clbits=0, params=[])}

### Perform HPO

In [3]:
from hyperparameter_optimization import HyperparameterOptimizer

Set path to the files specifying the RL agent and where to store the HPO results

In [4]:
path_agent_config = '/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/template_configurations/agent_config.yaml'
save_results_path = 'hpo_results'

In [5]:
from quantumenvironment import QuantumEnvironment

In [6]:
q_env = QuantumEnvironment(gate_q_env_config)

SparsePauliOp(['II', 'XX', 'YZ', 'ZY'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])


In [7]:
optimizer = HyperparameterOptimizer(q_env=q_env,
                                    path_agent_config=path_agent_config, 
                                    save_results_path=save_results_path, 
                                    log_progress=True,
                                    num_hpo_trials=2)
optimizer.optimize_hyperparameters()

 51%|█████     | 50/98 [00:13<00:13,  3.67it/s]

Fidelity History: []
SparsePauliOp(['II', 'XX', 'YY', 'ZZ'],
              coeffs=[ 0.25+0.j,  0.25+0.j, -0.25+0.j,  0.25+0.j])


 52%|█████▏    | 51/98 [00:13<00:12,  3.73it/s]

mean tensor([-0.1342,  0.9942,  0.3925,  0.2300, -0.2565,  0.2014, -0.9995])
Average return: 0.4786426082804319
DFE Rewards Mean: 0.4786426082804319
DFE Rewards standard dev 0.08479575070399677
Returns Mean: 0.6650793
Returns standard dev 0.16827853
Advantages Mean: -0.15827897
Advantages standard dev 0.16827853
Fidelity History: []
SparsePauliOp(['II', 'IZ', 'XI', 'XZ'],
              coeffs=[ 0.25+0.j, -0.25+0.j,  0.25+0.j, -0.25+0.j])


 53%|█████▎    | 52/98 [00:14<00:12,  3.67it/s]

...

mean tensor([-0.1433, -0.9831, -0.4276, -0.0282, -0.2645,  0.0739,  0.9960])
Average return: 0.4404762415296227
DFE Rewards Mean: 0.4404762415296227
DFE Rewards standard dev 0.07772887359414853
Returns Mean: 0.5904192
Returns standard dev 0.14041059
Advantages Mean: -0.030319642
Advantages standard dev 0.14041059
Fidelity History: []
SparsePauliOp(['II', 'XX', 'YY', 'ZZ'],
              coeffs=[ 0.25+0.j,  0.25+0.j,  0.25+0.j, -0.25+0.j])
mean tensor([-0.1313, -0.9791, -0.4831, -0.0558, -0.2373,  0.1418,  0.9933])
Average return: 0.4609760164580915
DFE Rewards Mean: 0.4609760164580915
DFE Rewards standard dev 0.0839312112440755
Returns Mean: 0.63017654
Returns standard dev 0.15671413
Advantages Mean: 0.05866526
Advantages standard dev 0.15671413
Fidelity History: []
SparsePauliOp(['II', 'XY', 'YX', 'ZZ'],
              coeffs=[ 0.25+0.j,  0.25+0.j, -0.25+0.j, -0.25+0.j])
mean tensor([-0.1769, -0.9861, -0.5739, -0.1259, -0.2483,  0.1334,  0.9960])
Average return: 0.46452991265886473
DFE

 86%|████████▌ | 24/28 [00:01<00:00, 23.34it/s]

SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])
mean tensor([-0.1038, -0.9312, -0.4238, -0.2090, -0.1308,  0.1166,  0.9514])
Average return: 0.7848736448255664
DFE Rewards Mean: 0.7848736448255664
DFE Rewards standard dev 0.07036264541774971
Returns Mean: 1.601016
Returns standard dev 0.38379776
Advantages Mean: 0.7712362
Advantages standard dev 0.3837978
Fidelity History: []


100%|██████████| 28/28 [00:01<00:00, 21.98it/s]
[I 2024-01-25 15:03:38,996] Trial 1 finished with value: 0.4963349345701975 and parameters: {'N_UPDATES': 28, 'N_EPOCHS': 12, 'MINIBATCH_SIZE': 64, 'BATCHSIZE_MULTIPLIER': 2, 'LR': 0.004247062447252421, 'GAMMA': 0.9791207939698907, 'GAE_LAMBDA': 0.9670613772766286, 'ENT_COEF': 0.0008484573907645138, 'V_COEF': 0.6212487024971437, 'GRADIENT_CLIP': 0.689859388192491, 'CLIP_VALUE_COEF': 0.27387145109536043, 'CLIP_RATIO': 0.23395881031735236}. Best is trial 0 with value: 0.6190536311091085.


SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[0.25+0.j, 0.25+0.j, 0.25+0.j, 0.25+0.j])
mean tensor([-0.0932, -0.9332, -0.3921, -0.2298, -0.1255,  0.1313,  0.9091])
Average return: 0.8187080175232382
DFE Rewards Mean: 0.8187080175232382
DFE Rewards standard dev 0.07102336046384021
Returns Mean: 1.8181818
Returns standard dev 0.5307631
Advantages Mean: 0.4203944
Advantages standard dev 0.5307631
Fidelity History: []
SparsePauliOp(['II', 'IZ', 'YI', 'YZ'],
              coeffs=[ 0.25+0.j, -0.25+0.j, -0.25+0.j,  0.25+0.j])
mean tensor([-0.0908, -0.9752, -0.4985, -0.2634, -0.2843,  0.1043,  0.8877])
Average return: 0.18869863808165044
DFE Rewards Mean: 0.18869863808165044
DFE Rewards standard dev 0.07070649318630991
Returns Mean: 0.21299955
Returns standard dev 0.088712
Advantages Mean: -1.6569061
Advantages standard dev 0.088712
Fidelity History: []
SparsePauliOp(['II', 'XY', 'YX', 'ZZ'],
              coeffs=[ 0.25+0.j,  0.25+0.j, -0.25+0.j, -0.25+0.j])
mean tensor([-0.065

#### Quick Summary of HPO Task

In [8]:
optimizer.target_gate

{'target_gate': Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]),
 'target_register': [0, 1]}

In [9]:
optimizer.hyperparams

['N_UPDATES',
 'N_EPOCHS',
 'MINIBATCH_SIZE',
 'BATCHSIZE_MULTIPLIER',
 'LR',
 'GAMMA',
 'GAE_LAMBDA',
 'ENT_COEF',
 'V_COEF',
 'GRADIENT_CLIP',
 'CLIP_VALUE_COEF',
 'CLIP_RATIO',
 'BATCHSIZE']

In [10]:
optimizer.num_hpo_trials

2

In [11]:
optimizer.best_hpo_configuration

{'best_avg_reward': 0.6190536311091085,
 'best_hyperparams': {'N_UPDATES': 98,
  'N_EPOCHS': 17,
  'MINIBATCH_SIZE': 96,
  'BATCHSIZE_MULTIPLIER': 10,
  'LR': 0.00042023154103451897,
  'GAMMA': 0.9670099769157185,
  'GAE_LAMBDA': 0.9087475540398442,
  'ENT_COEF': 0.00010745841175495839,
  'V_COEF': 0.3779768820449133,
  'GRADIENT_CLIP': 0.503131379582895,
  'CLIP_VALUE_COEF': 0.23786622162463758,
  'CLIP_RATIO': 0.17325524513585705}}